In [1]:
# Import the necessary libraries(make sure that TensorFlow is installed)
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Load a pre-trained model (e.g., VGG16) and modify the top layers for the classification task
input_shape = (32, 32, 3)
num_classes = 10

# Load the VGG16 model with pre-trained ImageNet weights, excluding the top fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))


In [3]:
# Add new top layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)


In [4]:
# Create a new model combining the base model and the new top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers in the base model so they won't be trained
for layer in base_model.layers:
    layer.trainable = False


In [5]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Prepare the dataset (e.g., CIFAR-10) and preprocess the images
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train, num_classes), to_categorical(y_test, num_classes)

In [6]:
# Data augmentation to improve model performance
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

In [8]:
# Train the model and evaluate its performance
batch_size = 30
epochs = 5

history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          steps_per_epoch=x_train.shape[0] // batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_accuracy}')


Epoch 1/5
1666/1666 ━━━━━━━━━━━━━━━━━━━━ 262s 157ms/step - accuracy: 0.5383 - loss: 1.3073 - val_accuracy: 0.5794 - val_loss: 1.2075
Epoch 2/5
1666/1666 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - accuracy: 0.5000 - loss: 1.2147 - val_accuracy: 0.5791 - val_loss: 1.2088
Epoch 3/5
1666/1666 ━━━━━━━━━━━━━━━━━━━━ 234s 141ms/step - accuracy: 0.5604 - loss: 1.2399 - val_accuracy: 0.5890 - val_loss: 1.1803
Epoch 4/5
1666/1666 ━━━━━━━━━━━━━━━━━━━━ 37s 22ms/step - accuracy: 0.6000 - loss: 1.2898 - val_accuracy: 0.5880 - val_loss: 1.1794
Epoch 5/5
1666/1666 ━━━━━━━━━━━━━━━━━━━━ 268s 161ms/step - accuracy: 0.5719 - loss: 1.2103 - val_accuracy: 0.5929 - val_loss: 1.1575
313/313 - 55s - 177ms/step - accuracy: 0.5929 - loss: 1.1575
Test accuracy: 0.5928999781608582
